In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm # import statsmodels 

/usr/local/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read in the data
df = pd.read_csv("./../data/changes.csv")

In [3]:
df.set_index('date', inplace=True)
print(df.shape)
df.head()

(7115050, 38)


,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-17,1.564000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-18,1.569417,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-19,1.578167,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-20,1.599000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
# try removing the averages so as to not include endogenous variables on the right hand side
df.drop(['eurusd', 'vehicles'], axis=1, inplace=True)
print(df.shape)
df.head()

(7115050, 36)


,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-17,1.564000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-18,1.569417,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-19,1.578167,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-20,1.599000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
# replace the oil prices for the last 30 days with the predictions
oil = pd.read_csv('./../data/linear_oil_yhat.csv')

last_30 = ['2015-11-12', '2015-11-13', '2015-11-14', '2015-11-15', '2015-11-16', '2015-11-17', '2015-11-18',
          '2015-11-19', '2015-11-20', '2015-11-21', '2015-11-22', '2015-11-23', '2015-11-24', '2015-11-25',
          '2015-11-26', '2015-11-27', '2015-11-28', '2015-11-29', '2015-11-30', '2015-12-01', '2015-12-02',
          '2015-12-03', '2015-12-04', '2015-12-05', '2015-12-06', '2015-12-07', '2015-12-08', '2015-12-09',
          '2015-12-10', '2015-12-10']

for index, date in enumerate(last_30):
    df.loc[date, 'rotterdam'] = oil['rot_yhat'][index]
    df.loc[date, 'brent'] = oil['brent_yhat'][index]
    df.loc[date, 'wti'] = oil['wti_yhat'][index]

In [6]:
df.sort_values(by=['station', 'num_days'], inplace=True)
df.head()

,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-17,1.564000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-18,1.569417,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-19,1.578167,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-20,1.599000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
# add lag variable
df['lag_e5gas'] = df['e5gas'].shift(1)
# set first day at each statoin equal to 0
df.loc[df['num_days'] == 0, 'lag_e5gas'] = 0
df.head()

,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15,lag_e5gas
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2014-05-17,1.564000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.536647
2014-05-18,1.569417,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.564000
2014-05-19,1.578167,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.569417
2014-05-20,1.599000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.578167


In [8]:
# now order needs to be by day rather than by station -- reorder by num_days
df.sort_values(by=['num_days', 'station'], inplace=True)
df.head()

,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15,lag_e5gas
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.537824,53.519798,10.0004,538.870667,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.588412,48.894600,10.0005,12108.902344,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.607947,49.911800,10.0018,270.597382,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.545250,49.793301,10.0023,3399.334473,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [9]:
# split into train and test sets
day_30 = df['num_days'].unique()[-30]
train = df.loc[df['num_days'] < day_30]
test = df.loc[df['num_days'] >= day_30]

In [10]:
train.head()

,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15,lag_e5gas
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.537824,53.519798,10.0004,538.870667,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.588412,48.894600,10.0005,12108.902344,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.607947,49.911800,10.0018,270.597382,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
2014-05-16,1.545250,49.793301,10.0023,3399.334473,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [11]:
# split into input and outputs
train_X, train_y = train[train.columns.difference(['e5gas', 'station'])], train['e5gas']
test_X, test_y = test[test.columns.difference(['e5gas', 'station'])], test['e5gas']
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(6743830, 35) (6743830,) (371220, 35) (371220,)


In [12]:
# fit model
X = sm.add_constant(train_X) ## let's add an intercept (beta_0) to our model

# Note the difference in argument order
model = sm.OLS(train_y, X).fit() ## sm.OLS(output, input)
# predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  e5gas   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.921
Method:                 Least Squares   F-statistic:                 2.261e+06
Date:                Sun, 29 Apr 2018   Prob (F-statistic):               0.00
Time:                        13:53:13   Log-Likelihood:             1.4352e+07
No. Observations:             6743830   AIC:                        -2.870e+07
Df Residuals:                 6743794   BIC:                        -2.870e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7396      0.001    688.287      0.000       0.737       0.742
aral           0.0317   3.17e-05   1000.634      0.000       0.032       0.032
autobahn       0.0428   6.49e-05    659.648      0.000       0.043       0.043
brent       2.013e-05   3.57e-06      5.633      0.000    1.31e-05    2.71e-05
dautobahn   -1.89e-08   1.49e-09    -12.690      0.000   -2.18e-08    -1.6e-08
esso           0.0105   4.34e-05    242.356      0.000       0.010       0.011
jet           -0.0076   5.57e-05   -137.308      0.000      -0.008      -0.008
lag_e5gas      0.1722      0.000   1153.155      0.000       0.172       0.173
latitude       0.0002   2.14e-05      7.598      0.000       0.000       0.000
longitude     -0.0008    1.5e-05    -52.760      0.000      -0.001      -0.001
num_days    9.826e-05   1.73e-07    567.714      0.000    9.79e-05    9.86e-05
rotterdam      0.7288      0.000   2453.034      0.000       0.728       0.729
shell          0.0299   3.49e-05    855.797      0.000       0.030       0.030
state_1        0.0019   5.48e-05     34.058      0.000       0.002       0.002
state_10      -0.0093   6.59e-05   -140.708      0.000      -0.009      -0.009
state_11       0.0008      0.000      7.119      0.000       0.001       0.001
state_12       0.0040      0.000     38.111      0.000       0.004       0.004
state_13      -0.0005      0.000     -4.493      0.000      -0.001      -0.000
state_14      -0.0108      0.000    -81.175      0.000      -0.011      -0.010
state_15       0.0029   9.37e-05     31.492      0.000       0.003       0.003
state_2       -0.0094      0.000    -69.419      0.000      -0.010      -0.009
state_3        0.0048      0.000     36.885      0.000       0.005       0.005
state_4       -0.0216      0.000   -107.251      0.000      -0.022      -0.021
state_5       -0.0168      0.000   -116.054      0.000      -0.017      -0.017
state_6       -0.0031   6.26e-05    -49.225      0.000      -0.003      -0.003
state_7       -0.0056   9.67e-05    -58.258      0.000      -0.006      -0.005
state_8       -0.0031      0.000    -21.317      0.000      -0.003      -0.003
state_9       -0.0086   7.45e-05   -115.254      0.000      -0.009      -0.008
total          0.0231   7.28e-05    318.109      0.000       0.023       0.023
weekday_1     -0.0004   4.15e-05     -9.558      0.000      -0.000      -0.000
weekday_2      0.0004   4.15e-05      8.856      0.000       0.000       0.000
weekday_3     -0.0018   4.16e-05    -43.123      0.000      -0.002      -0.002
weekday_4     -0.0022   4.16e-05    -53.763      0.000      -0.002      -0.002
weekday_5      0.0012   4.16e-05     28.059      0.000       0.001       0.001
weekday_6     -0.0010   4.15e-05    -24.513      0.000      -0.001      -0.001
wti            0.0010   3.48e-06    292.298      0.000       0.001       0.001
==

DW is lower with the lag added than without (1.2 vs 1.4)

In [13]:
# make a prediction, however each new day must update the var(t-1) for the next day, which is e5gas(t-1)
# so we need to do this one day at a time
# also need to check the last day

yhat_days = {}
days = test_X['num_days'].unique()
gas_before = []
for index, day in enumerate(days):
    print(index, day)
    df = test_X.loc[test_X['num_days'] == day]
    if len(gas_before) > 0:
        df['lag_e5gas'] = gas_before

    test_X_beta = sm.add_constant(test_X)
    yhat = model.predict(test_X_beta)
    yhat_days[index] = yhat
    gas_before = yhat

# convert yhat to numpy array
yhat = np.array([])
for key, value in yhat_days.items():
    yhat = np.append(yhat, value)

0 545
1 546


KeyboardInterrupt: 

In [16]:
# calculate RMSE
rmse = sqrt(mean_squared_error(test_y, yhat))
print('Test RMSE: %.4f' % rmse)

Test RMSE: 0.035


* no lag with and without changes: 0.035

In [64]:
# each day to make a graph of the error
ob = pd.DataFrame(inv_y)
pred = pd.DataFrame(inv_yhat)
pred[15] = round(pred[15])
pred[16] = round(pred[16])

In [45]:
rmse_days = []
days = []
for i in range(545,575):
    c_ob = ob[ob[16] == i][0]
    c_pred = pred[pred[16] == i][0]
    rmse_days.append(sqrt(mean_squared_error(c_ob, c_pred)))
    days.append(i)
    

In [50]:
rmse_days
days_df = pd.DataFrame()
days_df['rmse'] = rmse_days
days_df['date'] = last_30
days_df['date'] = pd.to_datetime(days_df['date'])

In [52]:
from bokeh.plotting import figure, output_file, show

output_file("line.html")

p = figure(plot_width=400, plot_height=400, x_axis_type='datetime')
p.yaxis.axis_label = 'Root Mean Squared Error'

# add a line renderer
p.line(days_df['date'], days_df['rmse'], line_width=2)

show(p)

In [62]:
print(ob[ob[16] == 546][0].mean())
print(pred[pred[16] == 546][0].mean())

1.3557571172714233
1.3972167226620094


In [68]:
gas_201 = pred[pred[15] == 201][0]

In [70]:
output_file("stuff.html")

p = figure(plot_width=400, plot_height=400, x_axis_type='datetime')
p.yaxis.axis_label = 'Gasoline Price'

# add a line renderer
p.line(days_df['date'], gas_201, line_width=2, color='green')

show(p)